In [7]:
%load_ext autoreload
%autoreload 2

In [2]:
from generate_data import generate_data
from Modules import Sequential, Linear, ReLU, Tanh, Sigmoid
from training import train_model_SGD, accuracy

In [10]:
N = 1000 # Number of samples in each dataset

train_input, train_target, test_input, test_target = generate_data(1000, normalize=True)

print(train_input.shape)
print(train_input.std())
print(train_input.mean())

torch.Size([1000, 2])
tensor(1.)
tensor(4.4510e-07)


In [40]:
l1 = Linear(2,25)
a1 = ReLU()
l2 = Linear(25, 25)
a2 = ReLU()
l3 = Linear(25, 25)
a3 = ReLU()
l4 = Linear(25, 2)
model = Sequential(l1, a1, l2, a2, l3, a3, l4)

losses = train_model_SGD(model, train_input, train_target, nb_epoch=200, 
                         learning_rate=1e-3, mini_batch_size=100, verbose=True)

The loss has diverged to infinity, you should maybe decrease the value of the learning rate or change one of the activation functions.


In [ ]:
l1 = Linear(2,25)
a1 = ReLU()
l2 = Linear(25, 25)
a2 = ReLU()
l3 = Linear(25, 25)
a3 = ReLU()
l4 = Linear(25, 2)
model = Sequential(l1, a1, l2, a2, l3, a3, l4)

losses = train_model_SGD(model, train_input, train_target, nb_epoch=200, 
                         learning_rate=1e-3, mini_batch_size=100, verbose=True)

In [28]:
accuracy(model, train_input, train_target)

tensor(0.7910)

In [29]:
accuracy(model, test_input, test_target)

tensor(0.7590)

In [3]:
import numpy as np
nb_rounds = 10
train_accuracies = np.zeros(nb_rounds)
test_accuracies = np.zeros(nb_rounds)
for i in range(0,nb_rounds):
    # Generate normalize training and testing dataset of size N = 1000
    N = 1000 # Number of samples in each dataset
    train_input, train_target, test_input, test_target = generate_data(N, normalize=True)

    # Builds a network with three hidden layer of size 25 using sequential Linear modules and activation functions
    l1 = Linear(2,25)
    a1 = ReLU()
    l2 = Linear(25, 25)
    a2 = Tanh()
    l3 = Linear(25, 25)
    a3 = Sigmoid()
    l4 = Linear(25, 2)
    model = Sequential(l1, a1, l2, a2, l3, a3, l4)

    # Train the model using MSE and logging the losses (verbose = True)
    losses = train_model_SGD(model, train_input, train_target, nb_epoch=300, learning_rate=1e-3, mini_batch_size=100, verbose=True)
    
    train_accuracies[i] = accuracy(model, train_input, train_target).item()
    test_accuracies[i] = accuracy(model, test_input, test_target).item()
    
train_mean, train_std = train_accuracies.mean(), train_accuracies.std()
test_mean, test_std = test_accuracies.mean(), test_accuracies.std()

Epoch: 0, Loss: 95.1224365234375, Accuracy: 0.792
Epoch: 5, Loss: 0.8566398620605469, Accuracy: 0.940
Epoch: 10, Loss: 0.6008001565933228, Accuracy: 0.960
Epoch: 15, Loss: 0.44756847620010376, Accuracy: 0.969
Epoch: 20, Loss: 0.37548285722732544, Accuracy: 0.971
Epoch: 25, Loss: 0.3250141739845276, Accuracy: 0.974
Epoch: 30, Loss: 0.285168319940567, Accuracy: 0.981
Epoch: 35, Loss: 0.25102922320365906, Accuracy: 0.981
Epoch: 40, Loss: 0.22483110427856445, Accuracy: 0.983
Epoch: 45, Loss: 0.20721293985843658, Accuracy: 0.984
Epoch: 50, Loss: 0.19359681010246277, Accuracy: 0.985
Epoch: 55, Loss: 0.18191812932491302, Accuracy: 0.986
Epoch: 60, Loss: 0.17097947001457214, Accuracy: 0.987
Epoch: 65, Loss: 0.16096994280815125, Accuracy: 0.989
Epoch: 70, Loss: 0.15146894752979279, Accuracy: 0.989
Epoch: 75, Loss: 0.1420581340789795, Accuracy: 0.989
Epoch: 80, Loss: 0.13326410949230194, Accuracy: 0.990
Epoch: 85, Loss: 0.12543648481369019, Accuracy: 0.991
Epoch: 90, Loss: 0.1185283362865448, Ac

In [6]:
import matplotlib.pyplot as plt
model_names = ['Training','Testing']
x_pos = np.arange(len(model_names))
means = [train_mean, test_mean]
stds = [train_std,  test_std]

# Build the plot
plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Accuracy')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names)
ax.set_title('Framework Accuracy')
ax.yaxis.grid(True)

for i, m in enumerate(means):
    ax.text(i-0.05, 0.65 , "{:0.3f}".format(m))

# Save the figure and show
plt.show()

fig.savefig('results.pdf',format='pdf',orientation='landscape')

<Figure size 1500x800 with 1 Axes>